In [ ]:
dir_read = '/Users/rachellehorwitz/Documents/VTAlert/data/filtered01/'
dir_read = '/Users/rachellehorwitz/Documents/VTAlert/over18_eicu/'

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
diagnosis_pos = pd.read_csv(dir_read + 'diagnosis_vte_ever.csv')
diagnosis_neg = pd.read_csv(dir_read + 'diagnosis_no_vte.csv')

patient_pos = pd.read_csv(dir_read + 'patient_vte_ever.csv')
patient_neg = pd.read_csv(dir_read + 'patient_no_vte_ever.csv')

In [ ]:
patient_pos[patient_pos['age']==91]

In [ ]:
patient_neg[patient_neg['age']==91]

In [ ]:
def convertAgeToNum(df_pt):
    # Replace '> 89' with '91'
    over89 = df_pt[df_pt['age']=='> 89']
    print(str(over89.shape[0]) + ' patientunitstayids with age > 89 out of ' + str(df_pt.shape[0]))
    df_pt['age'] = df_pt['age'].replace('> 89', '91').astype(int)
    return df_pt

In [ ]:
patient_pos = convertAgeToNum(patient_pos)
patient_pos['label'] = 1

patient_neg = convertAgeToNum(patient_neg)
patient_neg['label'] = 0

In [ ]:
patient_pos[patient_pos['age']==91]

In [ ]:
patient_pos[patient_neg['age']==91]

In [ ]:
%matplotlib notebook
p1 = sns.kdeplot(patient_neg['admissionweight'], shade=True, color='b')
p2 = sns.kdeplot(patient_pos['admissionweight'], shade=True, color='r')

In [ ]:
diagnosis_pos.shape[0]

In [ ]:
diagnosis_neg.shape[0]

In [ ]:
patient_pos.shape[0]

In [ ]:
patient_neg.shape[0]

In [ ]:
cols_origorder = patient_neg.columns[0:-1].tolist()
cols_reorder = ['label'] + cols_origorder

In [ ]:
patient_neg.reindex(columns=cols_reorder)
patient_pos.reindex(columns=cols_reorder)

In [ ]:
patient = pd.concat([patient_neg, patient_pos])
#patient = convertAgeToNum(patient)

In [ ]:
# Concatenate to get X and y
patient = pd.concat([patient_pos, patient_neg])

In [ ]:
# Drop patients who don't have height or weight
patient = patient[~patient['admissionweight'].isnull()]

In [ ]:
patient.shape[0]
patient.head(10)

In [ ]:
patient['patientunitstayid'].shape[0]

In [ ]:
patient['patientunitstayid'].unique().shape[0]

In [ ]:
# Define X
X = patient[['age', 'admissionweight']].copy()
y = np.array(patient['label'])
type(X.age[0])

In [ ]:
# Create labels for each class
y = patient['label'].copy()

In [ ]:
# Baseline if always predict 0: wrong only for when y is 1
N0_bl = patient[patient['label']==0].shape[0]
N1_bl = patient[patient['label']==1].shape[0]
N0_bl/(N0_bl + N1_bl)
print(str(N0_bl))
print(str(N1_bl))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
predictions = logisticRegr.predict(X_test)
score = logisticRegr.score(X_test, y_test)
print(score)

In [ ]:
# # Remove patientunitstayids that are over 350 kg (770 lbs)
# ptunitstayid_tooheavy = patient_neg[patient]